In [1]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import requests
import seaborn as sns

src = "https://covidtracking.com/api/states/daily"
src_data = requests.get(src).json()[::-1]

In [6]:
states = list(set([s["state"] for s in src_data]))

recent_days = 3

all_states = []
for state in states:
    data = [{ **s, "positive": s["positive"] if s["positive"] else 0 } for s in src_data if s["state"] == state]
    now = data[-1]["positive"]
    then = data[-1 - recent_days]["positive"]
    recent_increase_rate = ((now / then) - 1) / recent_days if then > 0 else 0
    all_states.append({
        "fips": data[0]["fips"],
        "state": state,
        "recent_increase_rate": recent_increase_rate * 100
        })

df = pd.DataFrame(all_states, columns=all_states[0].keys())

colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]
max = df["recent_increase_rate"].max()
min = df["recent_increase_rate"].min()
step = (max - min) / (len(colorscale) - 1)
endpts = [i * step for i in range(len(colorscale) - 1)]

fig = go.Figure(data=go.Choropleth(
    locations=df['state'],
    z=df['recent_increase_rate'].astype(float),
    locationmode='USA-states',
    colorscale='Blues',
    autocolorscale=False,
    marker_line_color='white', # line markers between states
    colorbar_title="% increase per day"
))

fig.update_layout(
    title_text='Recent rate of new infections',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

